In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import psycopg2 

In [ ]:
conn = psycopg2.connect(
    host="localhost",
    port=5432(default port),
    dbname="db_name",
    user="username",
    password="password"
)

In [3]:
px.defaults.template = "plotly_white"
base_layout = dict(height=520, title_x=0.02, font=dict(family="Inter, Arial", size=13))

In [ ]:
# Query: Top 20 Recipients by Gross Volume
query = """
SELECT payee_id AS recipient_id,
       COUNT(*) AS no_of_trans,
       SUM(amount) AS gross_volume
FROM transactions
GROUP BY payee_id
ORDER BY gross_volume DESC
LIMIT 20;
"""
df = pd.read_sql(query, conn)

fig = px.bar(
    df.sort_values('gross_volume', ascending = True),
    x = 'gross_volume', y = 'recipient_id',
    orientation = 'h',
    text = 'gross_volume',
    color = 'gross_volume',
    color_continuous_scale = px.colors.sequential.Turbo,
    labels = {'gross_volume':'Gross volume','recipient_id':'Recipient'}
)
fig.update_traces(texttemplate = '%{text:.2f}', textposition = 'outside')
fig.update_layout(title = "Top 20 Recipients by Gross Volume", margin = dict(l = 200))

/var/folders/nt/4rp641zs5lg7_234f6d1h4_00000gn/T/ipykernel_42775/2224705700.py:11: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [ ]:
# Query: Take Volume and Average Transactions by UPI apps
query = """
SELECT upi_app,
       COUNT(*) AS no_of_trans,
       SUM(amount) AS gross_volume,
       ROUND(AVG(amount),2) AS avg_trans
FROM transactions
GROUP BY upi_app
ORDER BY gross_volume DESC
LIMIT 30;
"""
df = pd.read_sql(query, conn)

fig = px.treemap(df, path = ['upi_app'], values = 'gross_volume',
                 color = 'avg_trans', color_continuous_scale = px.colors.sequential.Plasma,
                 hover_data = {'no_of_trans':True, 'avg_trans':True})
fig.update_layout(title = 'UPI Apps — Share of Volume(Colored by Transactions)')


/var/folders/nt/4rp641zs5lg7_234f6d1h4_00000gn/T/ipykernel_42775/949819054.py:12: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [23]:
# Query: Hour-of-day Activity
query = """
SELECT EXTRACT(hour FROM trans_ts) AS hour,
       COUNT(*) AS no_of_trans,
       SUM(amount) AS gross_volume
FROM transactions
GROUP BY hour
ORDER BY hour;
"""
df = pd.read_sql(query, conn)

fig = go.Figure()
fig.add_trace(go.Scatter(
    x = df['hour'], y=df['no_of_trans'],
    mode = 'lines+markers', line = dict(shape = 'spline', width = 3),
    marker=dict(size=6),
))
fig.update_layout(title = 'Transactions by Hour of Day', xaxis_title = 'Hour (0-23)', yaxis_title = 'Count')


/var/folders/nt/4rp641zs5lg7_234f6d1h4_00000gn/T/ipykernel_42775/2638765936.py:10: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [ ]:
# Query: Daily Anomaly Detection using Rolling z-score on Daily Volume
query = """
WITH daily AS (
  SELECT date_trunc('day', trans_ts)::date AS day, SUM(amount) AS daily_volume
  FROM transactions
  GROUP BY date_trunc('day', trans_ts)
),
stats AS (
  SELECT day, daily_volume,
         AVG(daily_volume) OVER (ORDER BY day ROWS BETWEEN 30 PRECEDING AND 1 PRECEDING) AS roll_mean,
         STDDEV(daily_volume) OVER (ORDER BY day ROWS BETWEEN 30 PRECEDING AND 1 PRECEDING) AS roll_sd
  FROM daily
)
SELECT day, ROUND(daily_volume,2) AS daily_volume, ROUND(roll_mean,2) AS roll_mean, ROUND(roll_sd,2) AS roll_sd,
       CASE WHEN roll_sd IS NOT NULL AND ABS(daily_volume - roll_mean) > 3 * roll_sd THEN 'ANOMALY' ELSE 'ok' END AS flag
FROM stats
ORDER BY day DESC
LIMIT 200;
"""
df = pd.read_sql(query, conn, parse_dates = ['day'])

# plot
d = df.sort_values('day')
fig = go.Figure()
fig.add_trace(go.Scatter(x = d['day'], y = d['daily_volume'], mode = 'lines', name = 'Daily volume', line = dict(width = 2, color = 'green')))
fig.add_trace(go.Scatter(x = d['day'], y = d['roll_mean'], mode = 'lines', name = 'Rolling mean', line = dict(width = 2, dash = 'dash', color = 'red')))
anoms = d[d['flag'] == 'ANOMALY']
fig.add_trace(go.Scatter( x = anoms['day'], y = anoms['daily_volume'], mode = 'markers', marker = dict(size = 10, color = 'red', symbol = 'x'), name = 'Anomalies'))
fig.update_layout(title = 'Daily Volume with Anomalies', xaxis_title = 'Day', yaxis_title = 'Daily volume')


/var/folders/nt/4rp641zs5lg7_234f6d1h4_00000gn/T/ipykernel_42775/1763007870.py:20: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [ ]:
# Query: Average Transaction and Transaction Frequency by Customer Age Bracket
query = """
WITH age_grp AS (
  SELECT u.user_id, u.cust_age AS age,
         COUNT(t.trans_id) AS no_of_trans,
         AVG(t.amount) AS avg_amount
  FROM users u
  JOIN transactions t ON u.user_id = t.payer_id
  WHERE u.cust_age IS NOT NULL
  GROUP BY u.user_id, u.cust_age
)
SELECT CASE
         WHEN age < 25 THEN 'under 25'
         WHEN age BETWEEN 25 AND 34 THEN '25-34'
         WHEN age BETWEEN 35 AND 44 THEN '35-44'
         WHEN age BETWEEN 45 AND 54 THEN '45-54'
         ELSE '55 plus' END AS age_bracket,
       ROUND(AVG(no_of_trans),2) AS avg_trans_per_user,
       ROUND(AVG(avg_amount),2) AS avg_trans_amount
FROM age_grp
GROUP BY age_bracket
ORDER BY age_bracket;
"""
df = pd.read_sql(query, conn)

fig1 = px.bar(df, x = 'age_bracket', y = 'avg_trans_per_user', title = 'Average Transactions per User by Age Bracket',
              color = 'age_bracket', color_continuous_scale = px.colors.sequential.Magma)
fig2 = px.bar(df, x = 'age_bracket', y = 'avg_trans_amount', title = 'Average Transaction Amount by Age Bracket',
              color = 'avg_trans_amount', color_continuous_scale = px.colors.sequential.Viridis)
fig1.show()
fig2.show()


/var/folders/nt/4rp641zs5lg7_234f6d1h4_00000gn/T/ipykernel_42775/1884723021.py:24: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

